In [ ]:
from __future__  import print_function  # Python 2/3 compatibility

from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection

graph = Graph()

remoteConn = DriverRemoteConnection('shared-opslab-neptune-instance-1.c6rb6dyvnrbg.us-east-2.neptune.amazonaws.com','g')

# remoteConn = database-1-instance-1.c1dqdb3eljzx.eu-west-1.neptune.amazonaws.com:8182/gremlin, 'g'
g = graph.traversal().withRemote(remoteConn)

In [ ]:
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.aiohttp.transport import AiohttpTransport
from gremlin_python.process.traversal import *
import os

port = 8182
server = 'shared-opslab-neptune.cluster-c6rb6dyvnrbg.us-east-2.neptune.amazonaws.com'

endpoint = f'wss://{server}:{port}/gremlin'

graph=Graph()

connection = DriverRemoteConnection(endpoint,'g',
                 transport_factory=lambda:AiohttpTransport(call_from_event_loop=True))

g = graph.traversal().withRemote(connection)

# print(g.V().limit(3).toList())

# results = (g.V().hasLabel('airport').
#              sample(10).
#               order().by('code').
#               local(__.values('code','city').fold()).
#               toList())

results = g.V().has('runways',gte(6)).values('code')

# Print the results in a tabular form with a row index
for i,c in enumerate(results,1):
    print("%2d %s" % (i,c))

connection.close()

In [ ]:
target = 'https://raw.githubusercontent.com/krlawrence/graph/master/sample-data/air-routes-latest-nodes.csv'
endpoint

In [ ]:
!yum install curl --user

In [ ]:
%%bash

!curl -X POST 'https://shared-opslab-neptune.cluster-c6rb6dyvnrbg.us-east-2.neptune.amazonaws.com:8182/loader' \
    -H 'Content-Type: application/json' -d '{
      "source" : "s3://sagemaker-shared-resources/graph-neptune/air-routes-latest-nodes.csv",
      "format" : "csv",
      "iamRoleArn" : "arn:aws:iam::857975912692:role/NeptuneLoadFromS3",
      "region" : "us-east-2",
      "failOnError" : "FALSE",
      "parallelism" : "MEDIUM",
      "updateSingleCardinalityProperties" : "FALSE",
      "queueRequest" : "TRUE"
        }'

In [ ]:
import gensim
from gensim.models import Word2Vec

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
raw = ['dayslucky333@gmail', 'dayslucky222@gmail', 'daysl.ucky222@gmail',
       'luckyday222@gmail.com',
       'dayslucky', 'ralph@gmail.com', 'john@gmail.com']

In [ ]:
data = [word_tokenize(r)[0] for r in raw]
data

In [ ]:
model_vec = gensim.models.Word2Vec([data], min_count=1, vector_size=3, window=1)

In [ ]:
model_vec.train([data], total_examples=3, epochs=5)

In [ ]:
model_vec.wv.similarity('dayslucky333', 'dayslucky222')

In [ ]:
vector = model_vec.wv['dayslucky']
model_vec.wv.most_similar('dayslucky', topn=4)

In [ ]:
similarity = np.zeros(len(data)* len(data)).reshape(len(data),-1)

In [ ]:
for ix, d in enumerate(data):
    for ixx, dd in enumerate(data):
        similarity[ix, ixx] = model_vec.wv.similarity(d, dd)

In [ ]:
pd.DataFrame(similarity, columns=data, index=data).apply(lambda x:x>0).to_numpy(int)

In [ ]:
edge_matrix = [[1,0,0,1], [1,0,0,1]]

In [ ]:
import gensim.downloader as api

In [ ]:
df['fraud_flag'] = df['fraud_flag'].apply(lambda x: 1 if x=='fraud' else 0)

In [ ]:
def build_graph_bipartite(df_input, graph_type=nx.Graph()):
    df = df_input.copy()
    mapping = {x:node_id for node_id,x in enumerate(set(df['email_address'].values.tolist() + df['customer_id'].values.tolist()))}
    df['from'] = df['customer_id'].apply(lambda x: mapping[x])
    df['to'] = df['email_address'].apply(lambda x: mapping[x])

    df = df[['from', 'to', 'fraud_flag']].groupby(['from','to'])\
        .agg({'fraud_flag': [('velocity','count'),('fraud','mean')]})

    df.columns = df.columns.get_level_values(1)
    df = df.reset_index()
    G = nx.from_edgelist(df[['from', 'to']].values, create_using=graph_type)
    nx.set_edge_attributes(G, {(int(x['from']),int(x['to'])):x['velocity'] for idx, x in df[['from', 'to', 'velocity']].iterrows()}, 'velocity')
    nx.set_edge_attributes(G, {(int(x['from']),int(x['to'])):x['fraud'] for idx, x in df[['from', 'to', 'fraud']].iterrows()}, 'fraud')
    return G, df

In [ ]:
G, df_out = build_graph_bipartite(testdf, graph_type=nx.Graph())

In [ ]:
df_out[df_out['velocity']==2].index

In [ ]:
df_out = df_out.reindex(G.nodes())

In [ ]:
data = V
adj = np.zeros(len(data)*len(data)).reshape(len(data),-1)

for ix, d in enumerate(data):
    for ixx, dd in enumerate(data):
        if dd == 'C' or d =='B':
            adj[ix, ixx] = 1
adj matplotlib
spring_pos = nx.spring_layout(G)
cmap = matplotlib.colors.ListedColormap(['lightgray', 'red'])
node_sizes = [100 if x>1 else 10 for x in df_out.velocity]
node_colors = [1 if x>1 else 0 for x in df_out.velocity]
edge_colors = ['r' if x==1 else 'g' for x in nx.get_edge_attributes(G, 'fraud').values()]
nx.draw_networkx(G, pos=spring_pos, node_size=node_sizes, cmap=cmap,
                 node_color=node_colors, edge_color=edge_colors, width=3, with_labels=False)
plt.show()

In [ ]:
G = nx.from_numpy_matrix(adj)